# Homework 3 - Find the perfect place to stay in Texas!

##### Work done by Musie Meressa Berhe, Ilaria Servadio, (Omid Ghamiloo).

Show which packages we used:

In [1]:
import csv #for read the data
import os
import glob
import pandas as pd # for work with data frame
import itertools as it
from itertools import islice
import string #We use it later for removing punctuation
import numpy as np
import json #JSON Library of Python can parse JSON string or file

# we import nltk library for preprocess the documents:
from nltk.corpus import words as nltk_words
from nltk.tokenize import sent_tokenize, word_tokenize,RegexpTokenizer 
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter, OrderedDict

import math
import enchant
import itertools as it

from geopy import distance
import heapq
from operator import itemgetter

# For a nice view of the output
from IPython.display import HTML, display
from tabulate import tabulate

## Step 1: Data

Importing Data with **read_csv()**

In [66]:
df = pd.read_csv('Airbnb_Texas_Rentals.csv')
df.head(n = 2) #head(n) return the first n rows

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...


We delete the column called **'Unnamed: 0'** from our dataframe because the row index does not have to be imported into the tsv files.
The content of the file should be the following:

`average_rate_per_night \t  bedrooms_count \t city \t date_of_listing \t description \t latitude\t longitude \t title \t url`

In [68]:
df = df.drop('Unnamed: 0', axis = 1) 
df.head(n = 2)

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...


In [4]:
len(df) #corresponds to the number of file

18259

## Step 2: Create documents

We create a .tsv file(where each column is separated by the tab) for **each record** of the dataset.
The number of lines in our 'Airbnb_Texas_Rentals.csv' file is **18259**, we could see this thanks to    *len(df)*. Therefore, we create 18259 tsv file.

In [5]:
#Read the file and create a single tsv file for each record.  
for i in range(len(df)):
    file = open('Doc_i/doc_'+ str(i+1) 
              + '.tsv','w', newline = '', encoding = 'utf-8') 
    for j in range(9): # 9 is the number of columns
        file.write('%s\t' %df.iloc[i,j])
    file.close()

**Details:**
- We store all the documents into an empty folder called *'Doc_i'* and this folder is created especially for them;
- str(i+1) : **i** is the dataframe index but we increase it onefold because in thi way the numbering of the documents start from 1 and not from 0;
- The name of each file is *doc_i.tsv*.

For example, if we open **doc_11245.tsv**, the content of the file should be the following:

`$400	3	Crystal Beach	February 2016	Redfish Retreat is new custom built spacious 3/2 home that sits 500 ft from the beach! It is beautifully furnished for a fun filled beach getaway! It comfortably sleeps 14. The open living area is spacious with well equipped kitchen and dining room.	29.472450991244997	-94.5953701751515	Redfish Retreat *close to the beach	https://www.airbnb.com/rooms/11529245?location=Bolivar%20Peninsula%2C%20TX`

## Step 3: Search Engine
As a first common step, we preprocess the documents by:
- **Removing stopwords:** if there are words that don’t really have any meaning to our text we can just remove them. This words have sense in our language but in data analysis they are useless;
- **Removing punctuation:** ! # $ % & \ ' () * + , - . / : ; < = > ? @ [ \\ ] ^ _ ` {|} ~
- **Stemming:** the idea is take from the word the root stem of the word. We do this because for a lot of items we are going to have different variations of word based on their stems or at least sort of fixes at the end but really the actual meaning of that word is unchanged.
- **Lemmatizing:** is a very similar operation to stemming but the end result is a real word, so, the end result will be generally some form of synonym or it might be the original word. So, you might end up with a very different word but it will be a similar word with the same meaning;
- **if not word.isdigit():** we do not take the numbers.

We know that lemmatization may have accept only words that are part of the English dictionary so we delete all the words of other languages.e been a risky choice. In fact, using lemmatizing the answer to the query is larger than when it is not used. We have chosen to do this operation because the result could be more satisfactory for the customer. We show our reasoning with an ***example***:

In English (and most languages), words appear in different forms. Look at these two sentences:

`A garden with cactus` -> lemmatizer.lemmatize("cactus")) return **cactus**

`A garden with cacti`  -> lemmatizer.lemmatize("cacti") return **cactus**

Both sentences talk about the noun cactus, but they are using different inflections. When working with text in a computer, it is helpful to know the base form of each word so that you know that both sentences are talking about the same concept. Otherwise the strings “cactus” and “cacti” look like two totally different words to a computer.


In [6]:
# Normalize the word. i.e. tokenize,stemming, lemmatization and removing punctuations
exclude = set(string.punctuation)
def wordNormlization(text):
    stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()
    Normalized = []
    tokenizer = RegexpTokenizer(r'\w+')
    word_list = tokenizer.tokenize(text.lower())
    Remove_stopwords = [stemmer.stem(lemmatizer.lemmatize(word)) for word in word_list if word not in stopwords.words('english')]
    for word in Remove_stopwords:
        if not word.isdigit() and len(word)>1:
            word = ''.join(ch for ch in word if ch not in exclude) #remove the punctuation
            Normalized.append(word)
    return Normalized

### 3.1) Conjunctive query
At this moment, we narrow out interest on the **description** and **title** of each document.

Now, what we do is:

1) `Read` all the documents inside the folder which contains all the 18259 tsv files;

2) `Preprocess` the discription and title for each file;

3) Create a `list` of all the words "clean";

4) Create a `dictionary` whose key is the name of the document and the words contained in it for the items.

In [8]:
words = []
dict_ = {}
for i in range(len(df)):
    file = open('Doc_i/doc_'+ str(i+1) 
              + '.tsv','r', encoding = 'utf-8') 
    file_name = 'doc_'+ str(i+1) + '.tsv' 
    list_ = file.read().split('\t')
    desc_and_title = list_[4] + list_[7]
    cleaning = wordNormlization(str(desc_and_title))
    words.append(cleaning)
    dict_[file_name] = cleaning
    file.close()
words = list(it.chain(*words)) # Convert a nested list into one-dimensional list

If you are interested in seeing the complete list of our words, open the file `'listwords.txt'` which is in the GitHub repository.

In [25]:
# Lets save the words list for future use
with open('listwords.txt', 'w') as filehandle:  
    json.dump(words, filehandle)
    
#To read the listword and save it to our words variable (we impor json)
# open listwords file for reading
with open('listwords.txt', 'r') as filehandle:  
    words = json.load(filehandle)

**CreateVocabulary(words)**: this function take in input all words, create the set and assign an ID to each term.

*Vocabulary = {term : termID}*

In [26]:
def CreateVocabulary(words):
    # vocabulary = {term : termID}
    Vocabulary = {}
    wordList = list(set(words))
    for ID, element in enumerate(wordList):
        Vocabulary.update({element : ID})
    return Vocabulary

In [27]:
voc = CreateVocabulary(words)

If you are interested in seeing the id associated with each word, open the file `'vocabulary.txt'` which is in the GitHub repository.

In [28]:
# Lets save the vocabulary for future use
with open('vocabulary.txt', 'w') as filehandle:  
    json.dump(voc, filehandle)

The total number of words is:

In [29]:
len(voc)

13836

### 3.1.1) Create your index!
*invertedIndex = {termID : doc}*

In [30]:
#Build invertedIndex
# invertedIndex = {termID : doc}
def invertedIndex(vocabulary):
    invertedIndex= {}
    for key,value in vocabulary.items():
        for i in value:
            if (voc[i] in invertedIndex):
                invertedIndex[voc[i]] = invertedIndex[voc[i]] + [key]
            else:
                invertedIndex[voc[i]] = [key]
    return invertedIndex

In [31]:
invertedDic = invertedIndex(dict_)

If you are interested in seeing the inverted index, open the file `'inverted.json'` which is in the GitHub repository.

In [23]:
#Save our invertedIndex file for future use.
with open('inverted.json', 'w') as fp:
    json.dump(invertedDic, fp)

### 3.1.2) Execute the query
Given a query, that we let the user enter.

In [19]:
query = str(input().split())

A beautiful and lovely room with garden !


In [20]:
qnorm = wordNormlization(query) #Normalize the query

Show how the query became after applying the function `wordNormalization(query)` which applies stemming, lemmatization and removing punctuations.

In [21]:
qnorm

['beauti', 'love', 'room', 'garden']

**What documents do we want?**
Since we are dealing with conjunctive queries (***AND***), each of the returned documents should contain all the words in the query. 
`AND <--> intersection`

We use set intersection to find the query words in all the documents. Our approach is:

- **First,** we store the documents that contain the first word.
- **Second,** we check the intersection of the first word with the second word. If we found that there is a common documents for each word, we store the common files into `inter` set and continue to find the intersection with the next word. In the meantime, if we don't find any intersection with the words; we stop looking for the document that contain other words. 

In [33]:
inter = set() #set with the documents that contain all the words in the query
switch = True
for word in qnorm:
    if word in voc:
        if (switch):
            inter = inter.union(invertedDic[voc[word]])
            switch = False
        if(len(inter) >0):
                inter = inter.intersection(set(invertedDic[voc[word]]))
        else:
            break

Show what are all the documents that contain all the words contained in the query:

In [34]:
len(inter)

9

### Final output of the query

In [45]:
my_dataframe = pd.DataFrame()
for file in inter:
    table = pd.read_table('Doc_i/'+ file,sep='\t', header = None,index_col = None)
    my_dataframe = my_dataframe.append(pd.DataFrame(table))

df1 = my_dataframe.rename(columns={0: 'average_rate_per_night', 1:'bedrooms_count', 2:'City',
                  3:'date_of_listing',4:'Description',5:'latitude',
                                                6:'longitude',7:'Title',8:'Url'})

df2 = df1[['Title','Description','City','Url']]
df2.drop_duplicates

display(HTML(tabulate(df2,showindex=False, tablefmt="html")))

Spacious Private Room and Bath,"Beautiful Mid-Century home in the heart of Denton surrounded by trees, community garden, and parks. We are two adventure-seeking, music and art appreciating, animal loving individuals who are opening our home to like-minded travelers.",Denton,https://www.airbnb.com/rooms/8064889?location=Corinth%2C%20TX
Beautiful queen bedroom in NW Austin,"My house is close to Lakeline Mall, highways and the toll road, restaurants, grocery stores, hike and bike trails, parks, Lake Travis. You’ll love my home because of the quiet, safe neighborhood, the light, the really comfy queen size bed, the sunny kitchen, the gardens, hammocks and hot tub. My place is good for couples, solo adventurers, business travelers, and families (with kids). I also have 2 other rooms so if you need more beds please check out my other listings too.",Austin,https://www.airbnb.com/rooms/16755710?location=Cedar%20Park%2C%20TX
Historic Mansion Downtown Ft Worth; Bass Hall; TCU,"Less than 2 miles from TCU, the Fort Worth Convention Center Botanical Gardens; Bass Hall, Magnolia Ave.\n\nA beautiful one-hundred year old historic mansion listed on the National Registry, one of Fort Worth's landmark houses. Stepping through the front doors is like stepping back in time to another more gracious era- Lovely period furnishings abound, high ceilings with hardwood floors beckon throughout, adorned with gorgeous Persian rugs, in spacious light-filled rooms, and crystal chandeliers.",Fort Worth,https://www.airbnb.com/rooms/8664808?location=Burleson%2C%20TX
Garden Bedroom,"Comfortable and private, the garden bedroom is ideal for a long weekend or festival stay. It features a private closet, a full-sized bed and a lovely view of the backyard. \n\nCommon spaces are here for your use as well. Cook a meal and stick to your travel budget, sit in the living room front window with a cup of coffee and watch the peacocks in the morning or enjoy the sunshine and garden from the back patio. \n\nIf being out and about in Austin is your goal, our neighborhood is central and there are many easy options for getting around the city. We have and love dogs. Please get in touch if you would like to bring yours!\n\nThere are so many things to love about our neighborhood. Not only are we super close to great parkland, bars, restaurants, coffee shops, food trailers and vintage shops, we're surrounded by huge live oaks and peacocks! The neighbors are open and friendly and have made us feel welcome from the start. Some of our favorite places include Skylark Lounge, Contigo, Dai Due, Sugar Mama's, Coldtowne Theater, The Work Horse, Foreign and Domestic, Drink Well, Kome, Tyson's Tacos, Room Service Vintage, Uptown Modern, Michi Ramen, Big Bertha's and Lucy's Fried Chicken. We also really dig our backyard patio and grill, and we think you will, too.\n\nOur neighborhood connects directly to the Mueller Greenway trails, putting guests less than a mile away from The Thinkery and Mueller Park (they have a great Farmer's Market on Sundays and a beautiful loop for walkers and runners alike). By bike, take the Greenway two miles west to UT, North Loop or Hyde Park, take it a mile southeast to Cherrywood or Manor Road, five miles further and you're in the thick of the East Side & downtown without ever having to leave a bike lane. If cycling is not your thing, shared rides are abundant (getme is a good app) as are Car To Go cars. There is also a bus stop less than two blocks from the house, and we are two miles from the Capmetro station at Highland. I-35 is literally a stone's throw from the house, putting you just a few exits from wherever you want to be in Austin. If you are interested, we can provide bike rental services and route recommendations.\n\nWe have two dogs, one big and one small. Although we are responsible owners and you will find the house clean, please consider this if you are afraid of dogs or have allergies. On the other hand, if you would like to bring your well-beh

##### Duplicate strategy:
If we read the airbnb policy on their site we read that publishing the ad is completely free.
But when the host receives a reservation, airbnb charges the hosts for the costs of the service.
So for airbnb if a host enters the ad multiple times does not change anything but the result of the customer search does not have to report duplicates.That's why we decided to delete them now and not before.

### 3.2) Conjunctive query & Ranking score
In the new Search Engine, given a query, we want to get the top-k documents related to the query.
### 3.2.1) Inverted index
In our second Inverted Index for each word we want the list of documents in which it is contained in, and the relative `tfIdf` score.

***Term Frequency `TF`***: is the number of times each word shows up in a particular document(text). We use `Counter()` function to count the number of occurence of each words in the document. To calculate the TF of a term, we divide the occurence of the word by length of the text. 
$$\huge tf_i,_j = \frac{n_i,_j}{\sum_kn_i,_j}$$

$$n_i,_j = \mathcal{number\, of\, times\, the\, word \,appears\, in \,a \,document}$$

$$\sum_kn_i,_j = \mathcal{the\, total\, number\, of\, words\, in \,that \,document}$$

In [12]:
def term_freq(term, text):
    termcountdic = Counter(text)
    return termcountdic[term] / len(text)

***Inverse Document Frequency `IDF`:*** is the total number of documents divided by the number of documents the term or word shows up in. The words that occur rarely in the corpus have a high IDF score. 
$$\huge idf(w) = \frac{N}{df_t}$$

In [13]:
with open('listwords.txt') as f:
    Allword = json.load(f)

In [14]:
wordDict = dict(Counter(Allword))
N = 18259
def idf(Allword,word):
    return math.log(N/float(wordDict[word]))

In [ ]:
wordDict

We build our InvertedIndex by scoring the TFIDF values for each term(word). `TFIDF` score (w) for a word in a document.

$$\huge w_i,_j = tf_i,_j \times\frac{N}{df_i}$$

$$tf_i,_j = \mathcal{number\, of\, occurrences\, of\, i \,in\, in \,j}$$

$$df_i= \mathcal{number\, of\, documents\, containing\, i}$$

$$N = \mathcal{ total \, number \, of\, documents}$$

In [15]:
def wordNorm(text):
    stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()

    Normalized = []
    tokenizer = RegexpTokenizer(r'\w+')
    word_list = tokenizer.tokenize(text.lower())
    Remove_stopwords = [stemmer.stem(lemmatizer.lemmatize(word)) for word in word_list if word not in stopwords.words('english')]
    
    d = enchant.Dict("en_US")
    for word in Remove_stopwords:
        if not word.isdigit() and len(word)>1 and d.check(word):#remove the punctuation
            Normalized.append(word)
    return Normalized

In [42]:
def invertedIndex(vocabulary,path):
    # invertedIndex = {termID : (doc, TF*IDF)}
    invertedIndex = {}
    counter = 0
    for file in glob.glob(path):
        df = pd.read_table(file,sep='\t',names=['average_rate_per_night','bedrooms_count',
                                                'city','date_of_listing',
                                                'description','latitude','longitude','title','url'])
       
        desc = wordNorm(str(list(df['description'])))
        title = wordNorm(str(list(df['title'])))
        for word in vocabulary:
                tf = term_freq(word,desc+title)
                if(tf>0):
                    if(vocabulary[word] in invertedIndex.keys()):
                        invertedIndex[vocabulary[word]] +=[(file,tf*idf(Allword,word))] #[(file,tfidf(word,file))]
                    else:
                        invertedIndex[vocabulary[word]] = [(file,tf*idf(Allword,word))]

    return invertedIndex

In [ ]:
invIndex = invertedIndex(voc,'readall/*')

In [16]:
with open('IFIDFinverted.json','r') as f:
    invIndex = json.load(f)

This intersection function return the common files (i.e. Conjuctive Query) with their corresponding **TFIDF score**.

In [17]:
def intersection(qnorm,invIndex,vocab,inter):
    #This intersection function return the common files with their corresponding TFIDF score
    IDs = []
    doclist = []
    tmp = []
    for word in qnorm:
        IDs.append(vocab[str(word)])
        
    for file in inter:
        for id in IDs:
            tmp = invIndex[str(id)]
            for files in tmp:
                if file == files[0]:
                    doclist.append(files)

    return doclist

To calculate the TF score of query, we count the occurence of the `word` and divide it by the summation of `all word occurences`.
To calculate the IDF score of query, we calculate the sum of squared word occurences and eventually return the square root of the summation. Additionaly, we created the `dot product function`, `document_length`and `query length functions` that will help us to find the cosine similarity between the query and whole documents. 
    $$ Cosine Similarity = \frac{Dot Product(Query,Document)}{||Query||*||Document||}$$  

In [18]:
def Q_tf(word,text):
    tf = Counter(text)
    return(tf[word]/sum(tf.values()))
def Q_idf(query):
    count = Counter(query)
    sumsquare = 0
    for i in count.values():
        sumsquare+= np.square(i)
    return np.sqrt(sumsquare)

def query_length(query):    
    count = Counter(query)
    sumsquare = 0
    for i in count.values():
        sumsquare+= np.square(i)
    return np.sqrt(sumsquare)

def dot_product(q,doc):
    sumofproduct = 0
    for i in q:
            sumofproduct += Q_tf(i,q)*Q_idf(q)*doc[1]
    return(sumofproduct)
def document_length(doc):
    sumsquare = 0
    for i in doc:
        sumsquare+=i[1]**2
    return np.sqrt(sumsquare)

### 3.2.2) Execute the query
Once we get the right set of documents, we want to know which are the most similar according to the query. For this purpose, as scoring function we will use the `Cosine Similarity` with respect to the tfIdf representations of the documents.

The input query entered by the user should be used to match the relevant documents and score them. 

In [19]:
query = str(input().split())
qnorm = wordNorm(query)
inter = set() #set with the documents that contain all the words in the query
switch = True
tmp = set()
sumTfIDF = 0
for word in qnorm:
    if word in voc:
        file_score = invIndex[str(voc[word])]
        if (switch):
            for file in file_score:
                inter.add(file[0])
            switch = False
        if(len(inter) >0):
            for file in file_score:
                tmp.add(file[0])
            inter = inter.intersection(tmp)
        else:
            break
inter = intersection(qnorm,invIndex,voc,inter)
cosine_similarity = {}
for i in inter:
    cosine_similarity[i[0]] = dot_product(qnorm,i)/(query_length(qnorm)*document_length(inter))
sortedResult = sorted(cosine_similarity.items(), key=lambda kv: kv[1],reverse=True)

topFiveResult = sortedResult[:5]
topFiveSimilarityScore = []
for i in topFiveResult:
    topFiveSimilarityScore.append(round(i[1],2))
Similarity = pd.Series(topFiveSimilarityScore)

my_dataframe = pd.DataFrame()
for file in topFiveResult:
    table = pd.read_table(str(file[0]),sep='\t', header = None,index_col = None)
    my_dataframe = my_dataframe.append(pd.DataFrame(table))
df1 = my_dataframe.rename(columns={0: 'average_rate_per_night', 1:'bedrooms_count', 2:'City',
                  3:'date_of_listing',4:'Description',5:'latitude',
                                                6:'longitude',7:'Title',8:'Url'})
df1["Similarity"] = Similarity.values
df2 = df1[['Title','Description','City','Url','Similarity']]
df2[:5]

My Dream is to go to Dallas


,Title,Description,City,Url,Similarity
0,Beautiful home by SeaWorld area,Cool comfortable bedroom but shared with other...,San Antonio,https://www.airbnb.com/rooms/12151340?location...,0.37
0,PrivateMasterBed/Bath NoExtra fees,LQQK @my reviews! (Other room)\nThis listing h...,Belton,https://www.airbnb.com/rooms/3669104?location=...,0.24
0,PrivateMasterBed/Bath NoExtra fees,LQQK @my reviews! (Other room)\nThis listing h...,Belton,https://www.airbnb.com/rooms/3669104?location=...,0.23
0,Resort Attractions' Bar Room Suite!,The mahogany bar sets the stage for fun! Enter...,Willis,https://www.airbnb.com/rooms/315458?location=C...,0.18
0,Solo world traveler private room,Traveling around the world solo is one of the ...,San Antonio,https://www.airbnb.com/rooms/15789425?location...,0.16


### Step 4: Define a new score!

A single user can give in input more information than the single textual query, in our case 
we ask the user to enter:
- the price he would like to spend per night,
- the number of rooms he needs 
- where he wants to go (we ask this through latitude and longitude).

In [46]:
query = str(input().split())
qnorm = wordNormlization(query) #Normalize the query

A beautiful and lovely room with garden !


In [47]:
room = int(input('Enter the number of rooms you need: '))

Enter the number of rooms you need: 3


In [48]:
price = int(input('Enter how much you would like to spend per night: '))

Enter how much you would like to spend per night: 34


In [49]:
latitude, longitude = (input().split(' '))
lat = float(latitude)
long = float(longitude)

32.892048 -96.871741


In [56]:
requests = [price, room, lat,long] #The user's request to which we must respond

As a starting point, get the query-related documents by exploiting the search engine of Step 3.1.:

In [29]:
inter = set() #set with the documents that contain all the words in the query
switch = True
for word in qnorm:
    if word in voc:
        if (switch):
            inter = inter.union(invertedDic[str(voc[word])])
            switch = False
        if(len(inter) >0):
                inter = inter.intersection(set(invertedDic[str(voc[word])]))
        else:
            break
my_dataframe = pd.DataFrame()
for file in inter:
    table = pd.read_table('Doc_i/'+ file,sep='\t', header = None,index_col = None)
    my_dataframe = my_dataframe.append(pd.DataFrame(table))
df1 = my_dataframe.rename(columns={0: 'average_rate_per_night', 1:'bedrooms_count', 2:'City',
                  3:'date_of_listing',4:'Description',5:'latitude',
                                                6:'longitude',7:'Title',8:'Url'})

In [53]:
df1['price'] = df1['average_rate_per_night'].replace({'\$':''}, regex = True)
# We remove Dollar Sign from the data frame column, in this way we can consider the values of
# df1['average_rate_per_night'] such as integer. We store this values in a new column.

Now we realize our **scoring function** that has as variable the price, the number of rooms needed and the place (as already mentioned above):

In [54]:
def scoring_function(requests):
    score = []
    for row in df1.itertuples():
        k = distance.distance((lat, long), (row.latitude, row.longitude)).km
        #We calculate the distance between the coordinates entered by the user and those of the hotel
        s = 0
        if k == 0: #Great! This place is right where the user is looking!
            s += 1 #For this, we add a point
        elif  1 <= k <= 50:
        #If the house is within a certain range of distance from the place requested by the user
            s += 0.5 #It does not take the maximum score
        else:
            s += 0.01 #Too far!
        if room == row.bedrooms_count: #Great! Just the number of rooms that the user was looking for!
            s += 1 #For this, we add one point
        elif room == (row.bedrooms_count-1) or room == (row.bedrooms_count + 1):
            s += 0.7 #This is not exactly what was requested but still goes well
        elif room == (row.bedrooms_count -2) or room == (row.bedrooms_count +2):
            s += 0.01 #They're not user's kind of folks
        if price == int(row.price): #Great! Exactly what was needed
            s += 0.7
        elif price < int(row.price): #wow...Even better! The price is lower
            s += 1 
        elif price > int(row.price): #Too expensive
            s += 0.01
        score.append(s)
    return score

In [57]:
all_scores = scoring_function(requests)

In [58]:
df1['Score'] = all_scores 
# We add a new column to the dataframe with the score

We use a **heap data structure** (we used the python `heapq` library) for maintaining the *top-k* documents.

In [59]:
ListofDataFrame = []
for i in df1.values:
    ListofDataFrame.append(i)
top_five = heapq.nlargest(5, ListofDataFrame, key=itemgetter(11))

In [60]:
heaped = pd.DataFrame(top_five,columns=['average_rate_per_night','bedrooms_count',
                         'City','date_of_listing','Description','latitude','longitude','Title',
                          'Url',9,'price','Score'])

In [36]:
top_K = heaped[['Title','Description','City','Url','Score']]
top_K #In our case K is 5

,Title,Description,City,Url,Score
0,Historic Mansion Downtown Ft Worth; Bass Hall;...,"Less than 2 miles from TCU, the Fort Worth Con...",Fort Worth,https://www.airbnb.com/rooms/8664808?location=...,2.20
1,Kid-friendly cosy-luxury home,"Luxurious 3200 sqft villa with 4 bedrooms, 3.5...",Allen,https://www.airbnb.com/rooms/13654931?location...,2.20
2,Kid-friendly cosy-luxury home,"Luxurious 3200 sqft villa with 4 bedrooms, 3.5...",Allen,https://www.airbnb.com/rooms/13654931?location...,2.20
3,Historic Mansion Downtown Ft Worth; Bass Hall;...,"Less than 2 miles from TCU, the Fort Worth Con...",Fort Worth,https://www.airbnb.com/rooms/8664808?location=...,2.20
4,"Ocean Side Resort, Comfy Beds, Stylish decor!","Gorgeous grounds, 3 pools, hot tub, 3/4 mile w...",Galveston,https://www.airbnb.com/rooms/14593192?location...,2.01
